In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../6m_mix.csv", nrows=1000)

In [3]:
df.head()

,text,summary,spell
0,Chây ì noojp phạt nguội .,Chây ì nộp phạt nguội .,"[0, 0, 1, 0, 0, 0]"
1,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,mây là á phương tiện vi phạm dc camera ( di dd...,Đây là các phương tiện vi phạm được camera ( d...,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
3,Phổ biến nhất là lỗi ỗ không đúng nơi quy định .,Phổ biến nhất là lỗi đỗ không đúng nơi quy định .,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14...","Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from utils import *

class PhoBertDataset(Dataset):
    def __init__(self, indexs,
                 token_ids=None,
                 data=None,
                 is_train: bool = True,
                 ):
        self.indexs = indexs
        self.n_errors = 2
        self.mask_token_id = 0
        self.is_train = is_train
        if is_train:
            self.token_ids = token_ids
            self.correct_token_ids = self.token_ids['correct_token_ids']
            self.n_words = self.token_ids['n_words']
            self.error_ids = self.token_ids['error_ids']
            self.label_errors = self.token_ids['label_errors']
            self.size_subwords = self.token_ids['size_subwords']
        else:
            self.data = data
            self.word_error_tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
            self.error_ids = tokenizer_ids(self.data.error_sentence, self.word_error_tokenizer)
            self.size_subwords = calculate_size_subword(self.data.error_sentence, self.word_error_tokenizer)

    def __len__(self):
        return len(self.indexs)

    def __getitem__(self, idx):
        if self.is_train:
            idx = self.indexs[idx]
        data = {'error_id': torch.tensor(self.error_ids[idx], dtype=torch.long)}
        data['attention_mask'] = torch.tensor([0] + [1] * (data['error_id'].size(0) - 2) + [0], dtype=torch.long)
        data['batch_split'] = self.size_subwords[idx]
        if self.is_train:
            data['label_error'] = torch.tensor(self.label_errors[idx])
            data['word_correction'] = torch.tensor(self.correct_token_ids[idx])
        return data


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

In [35]:
from datasets import Dataset as dts

In [42]:
dataset = dts.from_pandas(df)


In [43]:
dataset

Dataset({
    features: ['text', 'summary', 'spell'],
    num_rows: 1000
})

In [44]:
def mapping_tokenizer(examples):
    examples['correct_token_ids'] = tokenizer(examples['summary'])['input_ids'][1:-1]
    examples['error_ids'] = tokenizer(examples['text'])['input_ids']
    examples['label_errors'] = eval(examples['spell'])
    del examples['spell']
    examples['batch_split'] = calculate_size_subword([examples['text']], tokenizer)[0]
    return examples

dataset = dataset.map(mapping_tokenizer, batched=False, num_proc=8)

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [45]:
dataset[1]

{'text': "Hàng chục ngàn phương tiện bị ghi hình vi phạm luật giao thông ở TP . HCM , bị ' bêu tên ' nhưng chủ van không chịu nộp phạt .",
 'summary': "Hàng chục ngàn phương tiện bị ghi hình vi phạm luật giao thông ở TP . HCM , bị ' bêu tên ' nhưng chủ vẫn không chịu nộp phạt .",
 'correct_token_ids': [1383,
  781,
  1232,
  1607,
  5959,
  45,
  701,
  819,
  5237,
  2283,
  1252,
  574,
  2178,
  25,
  253,
  5,
  383,
  4,
  45,
  104,
  31742,
  221,
  104,
  51,
  286,
  74,
  17,
  551,
  1140,
  1133,
  5],
 'error_ids': [0,
  1383,
  781,
  1232,
  1607,
  5959,
  45,
  701,
  819,
  5237,
  2283,
  1252,
  574,
  2178,
  25,
  253,
  5,
  383,
  4,
  45,
  104,
  31742,
  221,
  104,
  51,
  286,
  4616,
  17,
  551,
  1140,
  1133,
  5,
  2],
 'label_errors': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0],
 'batch_split': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [28]:
tokenizer.tokenize('Chây ì nộp phạt nguội .')

['Ch@@', 'ây', 'ì', 'nộp', 'phạt', 'nguội', '.']